Здесь всё основное как бы. Используется кэш дир чтобы сохранять модельку локально, чтобы потом её не пришлось перекачивать заново и не тратить кучу времени в пустую.

In [4]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small",cache_dir="C:/huggingface_cache/", torch_dtype=torch_dtype, use_safetensors=True, use_cache=True)
processor = AutoProcessor.from_pretrained("openai/whisper-small",cache_dir="C:/huggingface_cache/", use_cache=True)

Получаем пайплайн, обязательно указываем модель и токенайзер с процессора который мы получили выше, без данного процесса ничего работать не будет уж точно.

In [10]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

На выходе мы получаем полный текст и чанки с текстом.

In [13]:
pipe("7.mp3")

{'text': ' В этом одном предложении Нрашин текст есть доля.',
 'chunks': [{'timestamp': (0.0, 2.8),
   'text': ' В этом одном предложении Нрашин текст есть доля.'}]}

In [18]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

In [19]:
import nltk
nltk.downloader.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [20]:
sia = SentimentIntensityAnalyzer()

Тут используется достаточно интересное решение, а точнее свод любого текста (не важно на каком языке) в английский. Таким образом мы получаем возможность обрабатывать тональность текста на абсолютно любом языке.

In [25]:
from deep_translator import GoogleTranslator
#Используем гугл переводчик для перевода
translated = GoogleTranslator(source='auto', target='en').translate("эа")

In [26]:
sia.polarity_scores(translated)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

# Дальше всё было упаковано в классы для удобного взаимодействия.

Готовый класс, при инициализации нужно указать путь до кэш папки (чтобы модель не перекачивалась заново), у класса есть функция predict tag которая должна выявлять разметку: Highlights, base, 18+, gray, black, но пока она определяет base и black. Есть возможность её расширить.

In [48]:
class tagClassModel:
    def __init__(self, cache_dir):
        from transformers import AutoModelForSequenceClassification, AutoTokenizer
        self.model = AutoModelForSequenceClassification.from_pretrained("KoalaAI/Text-Moderation",cache_dir=cache_dir)
        self.tokenizer = AutoTokenizer.from_pretrained("KoalaAI/Text-Moderation",cache_dir=cache_dir)
    def predict_tag(self, text):
        from deep_translator import GoogleTranslator
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        inputs = self.tokenizer(translated, return_tensors="pt")
        outputs = self.model(**inputs)
        logits = outputs.logits
        probabilities = logits.softmax(dim=-1).squeeze()
        id2label = self.model.config.id2label
        labels = [id2label[idx] for idx in range(len(probabilities))]
        label_prob_pairs = list(zip(labels, probabilities))
        label_prob_pairs.sort(key=lambda item: item[1], reverse=True)
        ok = 0
        sh = 0
        for label, probability in label_prob_pairs:
            if label == "SH":
                sh = float(probability)
            elif label == "OK":
                ok = float(probability)
        if sh > ok:
            return "Black"
        else:
            return "Base"

In [50]:
tcm = tagClassModel("C:/huggingface_cache/")

config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\huggingface_cache\models--KoalaAI--Text-Moderation. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

In [54]:
tcm.predict_tag("Жизнь прекрасна")

'Base'

Тональность текста

In [62]:
def getTonality(text):
    from nltk.sentiment import SentimentIntensityAnalyzer
    from tqdm.notebook import tqdm
    import nltk
    
    nltk.downloader.download('vader_lexicon')
    sia = SentimentIntensityAnalyzer()
    
    from deep_translator import GoogleTranslator
    translated = GoogleTranslator(source='auto', target='en').translate(text)
    return sia.polarity_scores(translated)

In [65]:
getTonality("тест тональности")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Транскрибация текста

In [66]:
class Transcribation:
    def __init__(self, cache_dir):
        import torch
        from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
        from datasets import load_dataset

        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

        self.model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small",cache_dir=cache_dir, torch_dtype=torch_dtype, use_safetensors=True, use_cache=True)
        self.processor = AutoProcessor.from_pretrained("openai/whisper-small",cache_dir=cache_dir, use_cache=True)
        self.pipe = pipeline(
            "automatic-speech-recognition",
            model=self.model,
            tokenizer=self.processor.tokenizer,
            feature_extractor=self.processor.feature_extractor,
            return_timestamps=True,
            torch_dtype=torch_dtype,
            device=device,
        )
    def audio2text(self, audio_path):
        return self.pipe(audio_path)

In [67]:
tc = Transcribation("C:/huggingface_cache/")

In [71]:
tc.audio2text("4.mp3")

{'text': ' тут тест на русском. Here is very strong text in English all right.',
 'chunks': [{'timestamp': (0.0, 1.0), 'text': ' тут тест на русском.'},
  {'timestamp': (1.0, 3.8),
   'text': ' Here is very strong text in English all right.'}]}